Experiments with pie charts and CustomJsTransform

In [1]:
    %load_ext literacy
    from bokeh import plotting, models; from coffeetools import coffee; import pandas as pd; plotting.output_notebook()

Loading BokehJS ...

In [2]:
    source = pd.util.testing.makeDataFrame().sum().to_frame('x').abs().pipe(plotting.ColumnDataSource)

    source = pd.util.testing.makeDataFrame().sum().to_frame('x').abs().pipe(plotting.ColumnDataSource)

In [3]:
    def T(field, code="""return xs;""", **args):
        return dict(field=field, transform=models.CustomJSTransform(args=args, v_func=coffee.compile(code, bare=True)))

    def T(field, code="""return xs;""", **args):
        return dict(field=field, transform=models.CustomJSTransform(args=args, v_func=coffee.compile(code, bare=True)))

In [6]:
A simple exmaple using customjstransform to modify datasources with coffeescript.

    p = plotting.figure()
    p.add_glyph(
        source, 
        models.AnnularWedge(
            x=0, y=T('x', """return xs.map ()-> 0"""),
            end_angle=T('x', """return xs.map (v,i)-> i/xs.length*2*3.14159"""),
            start_angle=T('x', """return xs.map (v,i)-> (i+1)/xs.length*2*3.14159"""),
            outer_radius=T('x', """return xs.map (v)->1"""),
        )
    )
    plotting.show(p)

A simple exmaple using customjstransform to modify datasources with coffeescript.

    p = plotting.figure()
    p.add_glyph(
        source, 
        models.AnnularWedge(
            x=0, y=T('x', """return xs.map ()-> 0"""),
            end_angle=T('x', """return xs.map (v,i)-> i/xs.length*2*3.14159"""),
            start_angle=T('x', """return xs.map (v,i)-> (i+1)/xs.length*2*3.14159"""),
            outer_radius=T('x', """return xs.map (v)->1"""),
        )
    )
    plotting.show(p)

In [8]:
Pie charts suck.  They are biased.  But if they didn't have an orientation they would suck less.

    p = plotting.figure()
    r = p.add_glyph(
        source, 
        models.AnnularWedge(
            x=0, y=T('x', """return xs.map ()-> 0"""),
            end_angle=T('x', """
            dx = Math.max 1, Math.abs(xx.start-xx.end)/2/1.5
            total = xs.reduce (p,n)->p+n
            current = ((Math.log Math.abs(xx.start-xx.end)/10)%1)*dx*2*3.14159            
            return xs.map (v, i)-> 
                if i > 0
                    current += xs[i-1]/total*2*3.14159
                current
            """, xx=p.x_range),
            start_angle=T('x', """
            dx = Math.max 1, Math.abs(xx.start-xx.end)/2/1.5
            total = xs.reduce (p,n)->p+n
            current = ((Math.log Math.abs(xx.start-xx.end)/10)%1)*dx*2*3.14159
            return xs.map (v, i)-> 
                current += v/total*2*3.14159 
                current
            """, xx=p.x_range),
            outer_radius=T('x', """
            dx = Math.max 1, Math.abs(xx.start-xx.end)/2/1.5
            return xs.map (v)-> dx
            """, xx=p.x_range),
            inner_radius=T('x', """
            dx = .1*Math.max 1, Math.abs(xx.start-xx.end)/2/1.5
            return xs.map (v)-> dx
            """, xx=p.x_range),
            fill_alpha=.6
        )
    )
    p.x_range.callback = p.y_range.callback = models.CustomJS(args=dict(source=source), code="""
    source.trigger('change');
    dx = Math.abs(cb_obj.start - cb_obj.end)/2
    cb_obj.start = dx
    cb_obj.end = -dx
    """)
    plotting.show(p)
    
This pie chart doesn't suck if you enable wheel zoom.

Pie charts suck.  They are biased.  But if they didn't have an orientation they would suck less.

    p = plotting.figure()
    r = p.add_glyph(
        source, 
        models.AnnularWedge(
            x=0, y=T('x', """return xs.map ()-> 0"""),
            end_angle=T('x', """
            dx = Math.max 1, Math.abs(xx.start-xx.end)/2/1.5
            total = xs.reduce (p,n)->p+n
            current = ((Math.log Math.abs(xx.start-xx.end)/10)%1)*dx*2*3.14159            
            return xs.map (v, i)-> 
                if i > 0
                    current += xs[i-1]/total*2*3.14159
                current
            """, xx=p.x_range),
            start_angle=T('x', """
            dx = Math.max 1, Math.abs(xx.start-xx.end)/2/1.5
            total = xs.reduce (p,n)->p+n
            current = ((Math.log Math.abs(xx.start-xx.end)/10)%1)*dx*2*3.14159
            return xs.map (v, i)-> 
                current += v/total*2*3.14159 
                current
            """, xx=p.x_range),
            outer_radius=T('x', """
            dx = Math.max 1, Math.abs(xx.start-xx.end)/2/1.5
            return xs.map (v)-> dx
            """, xx=p.x_range),
            inner_radius=T('x', """
            dx = .1*Math.max 1, Math.abs(xx.start-xx.end)/2/1.5
            return xs.map (v)-> dx
            """, xx=p.x_range),
            fill_alpha=.6
        )
    )
    p.x_range.callback = p.y_range.callback = models.CustomJS(args=dict(source=source), code="""
    source.trigger('change');
    dx = Math.abs(cb_obj.start - cb_obj.end)/2
    cb_obj.start = dx
    cb_obj.end = -dx
    """)
    plotting.show(p)
    
This pie chart doesn't suck if you enable wheel zoom.